## Basic setting up

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy
import datetime

In [2]:
listings = pd.read_csv("barcelona_listings.csv")

In [3]:
reviews = pd.read_csv("barcelona_reviews.csv")

In [4]:
calendar = pd.read_csv("barcelona_calendar.csv")

In [5]:
engine = create_engine("sqlite:///project.db")

In [65]:
def null_count(x):
    return listings[x].isna().sum()# / len(listings[x])

def unique_count(x):
    return listings.nunique()# / listings.shape[0] * 100

#for i,c in enumerate(listings.columns):
 #   print(c, unique_count(c)[i])

## Import accommodates

In [7]:
df = pd.DataFrame(listings.accommodates.unique(), columns=["accommodates"])

In [9]:
df.to_sql("Accommodation", engine, if_exists="append", index=False, dtype={"accommodates" : sqlalchemy.INT})

In [ ]:
engine.execute("select count(*) from Accommodation ").fetchall()

## Import amenities

In [83]:
amenities = pd.DataFrame(listings.amenities.unique(), columns=["amenities"])

In [84]:
temp = []
for index, row in amenities.iterrows():
    temp.append(row["amenities"].split(','))

In [85]:
flat_list = []
for sublist in temp:
    for item in sublist:
        flat_list.append(item)

In [54]:
def remove_chars(item):
    for c in item:
        if c == "{" or c == "}" or c == "'" or c == '"' or c == "[" or c == "]":
            item = item.replace(c, "")
        item = item.strip().lower()
    return item

In [87]:
new_list = []
for item in flat_list:
    new_list.append('"' + remove_chars(item) + '"')
    

In [88]:
new_list = list(set(new_list))

In [90]:
new_list.remove('""')

In [91]:
new_list

['"espresso machine"',
 '"handheld shower head"',
 '"carbon monoxide detector"',
 '"stair gates"',
 '"elevator"',
 '"pets live on this property"',
 '"baby monitor"',
 '"ski-in/ski-out"',
 '"disabled parking spot"',
 '"building staff"',
 '"coffee maker"',
 '"private pool"',
 '"smart lock"',
 '"gym"',
 '"day bed"',
 '"accessible-height bed"',
 '"warming drawer"',
 '"shared pool"',
 '"microwave"',
 '"cooking basics"',
 '"wheelchair accessible"',
 '"pack ’n play/travel crib"',
 '"electric profiling bed"',
 '"free street parking"',
 '"balcony"',
 '"pillow-top mattress"',
 '"bidet"',
 '"air conditioning"',
 '"hbo go"',
 '"baby bath"',
 '"washer"',
 '"private living room"',
 '"single level home"',
 '"beachfront"',
 '"outdoor parking"',
 '"netflix"',
 '"dishes and silverware"',
 '"bath towel"',
 '"cat(s)"',
 '"sun loungers"',
 '"table corner guards"',
 '"changing table"',
 '"bbq grill"',
 '"dishwasher"',
 '"luggage dropoff allowed"',
 '"memory foam mattress"',
 '"free parking on premises"',
 '

In [92]:
amenities = pd.DataFrame(new_list, columns=["amenities"])

In [93]:
amenities.to_sql("Amenities", engine, if_exists="append", index=False, dtype={"amenities" : sqlalchemy.CHAR(32)})

In [24]:
listings.id.count()

13095

In [29]:
len(listings["amenities"].unique() == listings.amenities.unique())

11837

## Import Bathrooms

In [61]:
bathrooms = pd.DataFrame(listings.bathrooms.unique(), columns=["bathrooms"])

In [62]:
bathrooms = bathrooms.dropna()

In [63]:
bathrooms.to_sql("Bathrooms", engine, if_exists="append", index=False, dtype={"bathrooms" : sqlalchemy.FLOAT})

## Import Description

In [10]:
description = pd.DataFrame({"description_id": np.arange(len(listings.description)), "summary": listings.summary, "space": listings.space, "description": listings.description, "neighborhood_overview": listings.neighborhood_overview,"notes":listings.notes,"transit":listings.transit,"access":listings.access,"picture_url":listings.picture_url,"square_feet": listings.square_feet, "id": listings.id})

In [11]:
description["space"].fillna("",inplace=True)
description["neighborhood_overview"].fillna("",inplace=True)
description["description"].fillna("",inplace=True)
description["summary"].fillna("",inplace=True)
description["notes"].fillna("",inplace=True)
description["transit"].fillna("",inplace=True)
description["access"].fillna("",inplace=True)
description["picture_url"].fillna("",inplace=True)
description["square_feet"].fillna(-1,inplace=True)


In [14]:
description.to_sql("Described_Description", engine, if_exists="append", index=False, dtype={"description_id": sqlalchemy.INT, "summary": sqlalchemy.VARCHAR(1024), "space": sqlalchemy.VARCHAR(1024), "description": sqlalchemy.VARCHAR(1024), "neighborhood_overview": sqlalchemy.VARCHAR(1024),"notes":sqlalchemy.VARCHAR(1024),"transit": sqlalchemy.VARCHAR(1024),"access": sqlalchemy.VARCHAR(1024),"picture_url":sqlalchemy.CHAR(128),"square_feet": sqlalchemy.FLOAT, "id": sqlalchemy.INT})

## Import bedding

In [19]:
bedding = pd.DataFrame({"beds": listings.beds, "bed_type": listings.bed_type})

In [21]:
bedding = bedding.fillna(0).drop_duplicates()


In [22]:
bedding.to_sql("Bedding", engine, if_exists="append", index=False, dtype={"beds" : sqlalchemy.FLOAT, "bed_type" : sqlalchemy.CHAR(32)})

## Import Bedrooms

In [4]:
bedrooms = pd.DataFrame({"bedrooms": listings.bedrooms})

In [10]:
bedrooms = bedrooms.dropna().drop_duplicates()

In [11]:
bedrooms

,bedrooms
0,2.0
1,3.0
2,1.0
12,4.0
77,0.0
97,5.0
261,16.0
308,6.0
1436,12.0
2312,8.0


In [12]:
bedrooms.astype(int)

,bedrooms
0,2
1,3
2,1
12,4
77,0
97,5
261,16
308,6
1436,12
2312,8


In [13]:
bedrooms.to_sql("Bedrooms", engine, if_exists="append", index=False, dtype={"bedrooms" : sqlalchemy.INT})

## Import Room

In [14]:
room = pd.DataFrame({"room_type": listings.room_type})

In [17]:
room = room.drop_duplicates()

In [18]:
room.to_sql("Room", engine, if_exists="append", index=False, dtype={"room_type" : sqlalchemy.CHAR(32)})

## Import Property

In [19]:
prop = pd.DataFrame({"property_type": listings.property_type})

In [23]:
prop = prop.drop_duplicates()

In [24]:
prop.to_sql("Property", engine, if_exists="append", index=False, dtype={"propery_type" : sqlalchemy.CHAR(32)})

## Import Reviewer

In [29]:
reviewer = pd.DataFrame({"reviewer_id": reviews.reviewer_id, "reviewer_name" : reviews.reviewer_name})

In [50]:
reviewer = reviewer.drop_duplicates().drop(174201)

In [51]:
reviewer.to_sql("Reviewer", engine, if_exists="append", index=False, dtype={"reviewer_id" : sqlalchemy.INT, "reviewer_name" : sqlalchemy.CHAR(32)})

In [49]:
reviewer[reviewer.duplicated(['reviewer_id'], keep=False)]

,reviewer_id,reviewer_name
174201,6148697,Casa Nuna
179938,6148697,Mi Casa Bali


## Import Reviewed TODO

In [94]:
reviewed = pd.DataFrame({"id": reviews.listing_id, "reviewer_id" : reviews.reviewer_id, "comments": reviews.comments, "review_date" : reviews.date})

In [95]:
reviewed

,id,reviewer_id,comments,review_date
0,18666,1559265,Very nice flat. We had easy transportation to ...,2015-10-10
1,18674,4841196,"Great location. Clean, spacious flat. Would re...",2013-05-27
2,18674,11600277,Mi mejor recomendación para este departamento....,2014-03-02
3,18674,35231385,"Big apartment, well equipped.\nVery good servi...",2015-08-04
4,18674,23223644,The Check in was fast and flexible. The price ...,2016-06-20
5,18674,4756672,Great location and enough space in the apartme...,2018-06-18
6,21605,57647372,Meritxell était très accueillante et nous a mi...,2016-05-08
7,21605,62165990,Con mi esposa nos hospedamos en casa de Michae...,2016-05-22
8,21605,67778905,Even though the hosts were not there at the mo...,2016-05-27
9,21605,64869468,La habitación es perfecta para una pareja y la...,2016-05-29


In [100]:
reviewed[reviewed.duplicated(['id', "review_date","reviewer_id"], keep=False)]

,id,reviewer_id,comments,review_date
134162,1996730,85117476,"El apartamiento es increible, muy espacioso y ...",2016-07-23
134163,1996730,85117476,La segunda vez que me quedo en su piso y como ...,2016-07-23
210431,3823973,24495890,The reservation was canceled 17 days before ar...,2015-03-08
210796,3823973,24495890,The reservation was canceled 24 days before ar...,2015-03-08
212882,3823973,24495890,The reservation was canceled 10 days before ar...,2015-03-08
263426,6449766,57829170,The host canceled this reservation 24 days bef...,2018-06-05
263427,6449766,57829170,The host canceled this reservation 33 days bef...,2018-06-05
378123,16347842,19641299,The host canceled this reservation 11 days bef...,2017-11-16
378124,16347842,19641299,The host canceled this reservation 7 days befo...,2017-11-16


#### We need to change the primary key to (id, reviewer_id, date) and decide what to do with the duplicates

## Import Pricing

In [118]:
pricing = pd.DataFrame({"price": listings.price, "weekly_price" : listings.weekly_price, "monthly_price": listings.monthly_price, "security_deposit" : listings.security_deposit, "cleaning_fee" : listings.cleaning_fee, "guests_included" : listings.guests_included, "extra_people" : listings.extra_people, "minimum_nights" : listings.minimum_nights, "maximum_nights" : listings.maximum_nights, "id" : listings.id})

In [119]:
for i in pricing.index:
    pricing.at[i, "price"] = float(str(pricing.at[i, "price"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "weekly_price"]):
        pricing.at[i, "weekly_price"] = float(str(pricing.at[i, "weekly_price"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "monthly_price"]):
        pricing.at[i, "monthly_price"] = float(str(pricing.at[i, "monthly_price"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "security_deposit"]):
        pricing.at[i, "security_deposit"] = float(str(pricing.at[i, "security_deposit"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "cleaning_fee"]):
        pricing.at[i, "cleaning_fee"] = float(str(pricing.at[i, "cleaning_fee"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "extra_people"]):
        pricing.at[i, "extra_people"] = float(str(pricing.at[i, "extra_people"])[1:].replace(",", ""))

In [120]:
for i in pricing.index:
    if pd.isnull(pricing.at[i, "weekly_price"]):
        pricing.at[i, "weekly_price"] = pricing.at[i, "price"] * 7
    if pd.isnull(pricing.at[i, "monthly_price"]):
        pricing.at[i, "monthly_price"] = pricing.at[i, "price"] * 30
    if pd.isnull(pricing.at[i, "security_deposit"]):
        pricing.at[i, "security_deposit"] = 0
    if pd.isnull(pricing.at[i, "cleaning_fee"]):
        pricing.at[i, "cleaning_fee"] = 0

In [123]:
pricing.to_sql("Pricing", engine, if_exists="append", index=False, dtype={"price": sqlalchemy.FLOAT, "weekly_price" : sqlalchemy.FLOAT, "monthly_price": sqlalchemy.FLOAT, "security_deposit" : sqlalchemy.FLOAT, "cleaning_fee" : sqlalchemy.FLOAT, "guests_included" : sqlalchemy.INT, "extra_people" : sqlalchemy.FLOAT, "minimum_nights" : sqlalchemy.INT, "maximum_nights" : sqlalchemy.INT, "id" : sqlalchemy.INT})

## Import Verification

In [50]:
verification = pd.DataFrame(listings.host_verifications.unique(), columns=["host_verifications"])

In [51]:
temp = []
for index, row in verification.iterrows():
    temp.append(row["host_verifications"].split(','))

In [52]:
flat_list = []
for sublist in temp:
    for item in sublist:
        flat_list.append(item)

In [55]:
new_list = []
for item in flat_list:
    new_list.append('"' + remove_chars(item) + '"')

In [56]:
new_list = list(set(new_list))

In [57]:
new_list.remove('""')

In [58]:
new_list

['"manual_online"',
 '"sesame"',
 '"manual_offline"',
 '"photographer"',
 '"sent_id"',
 '"email"',
 '"offline_government_id"',
 '"sesame_offline"',
 '"kba"',
 '"weibo"',
 '"reviews"',
 '"zhima_selfie"',
 '"identity_manual"',
 '"government_id"',
 '"facebook"',
 '"google"',
 '"selfie"',
 '"work_email"',
 '"jumio"',
 '"phone"']

In [59]:
verification = pd.DataFrame({"host_verifications" : new_list})

In [61]:
verification.to_sql("Verification", engine, if_exists="append", index=False, dtype={"host_verifications" : sqlalchemy.CHAR(32)})

## Import verified_by TODO

## Import Calendar

In [40]:
cal = pd.DataFrame({"date" : calendar.date.unique()})

In [41]:
for i in cal.index:
    cal.at[i, "date"] = datetime.datetime.strptime(str(cal.at[i, "date"]), "%Y-%m-%d").date()

In [42]:
cal.to_sql("Calendar", engine, if_exists="append", index=False, dtype={"date" : sqlalchemy.DATE})

## Import Available_at

In [43]:
avail = pd.DataFrame({"id" : calendar.listing_id, "date" : calendar.date, "price" : calendar.price, "available" : calendar.available})

In [48]:
for i in calendar.index:
    avail.at[i, "date"] = datetime.datetime.strptime(str(calendar.at[i, "date"]), "%Y-%m-%d").date()

In [44]:
for i in avail.index:
    if not pd.isna(avail.at[i, "price"]):
        avail.at[i, "price"] = float(str(avail.at[i, "price"]).replace(",","").replace("$", ""))

In [45]:
avail = avail.fillna(-1)

In [49]:
avail.to_sql("Available_at", engine, if_exists="append", index=False, dtype={"id" : sqlalchemy.INT, "date" : sqlalchemy.DATE, "price" : sqlalchemy.INT, "available" : sqlalchemy.CHAR(1)})

## Import Policy

In [28]:
reduced = pd.DataFrame({"cancellation_policy" : listings["cancellation_policy"]})

In [34]:
reduced = reduced.drop_duplicates()

In [35]:
reduced.to_sql("Policy", engine, if_exists="append", index=False, dtype={"cancellation_policy" : sqlalchemy.CHAR(32)})

## Import Score

In [70]:
score = pd.DataFrame({"review_scores_rating" : listings.review_scores_rating, "review_scores_accuracy" : listings.review_scores_accuracy, "review_scores_cleanliness" : listings.review_scores_cleanliness, "review_scores_checkin" : listings.review_scores_checkin, "review_scores_communication" : listings.review_scores_communication, "review_scores_location" : listings.review_scores_location, "review_scores_value" : listings.review_scores_value, "id" : listings.id})

In [71]:
score = score.fillna(-1)

In [72]:
score.to_sql("Score", engine, if_exists="append", index=False, dtype={"review_scores_rating" : sqlalchemy.INT, "review_scores_accuracy" : sqlalchemy.INT, "review_scores_cleanliness" : sqlalchemy.INT, "review_scores_checkin" : sqlalchemy.INT, "review_scores_communication" : sqlalchemy.INT, "review_scores_location" : sqlalchemy.INT, "review_scores_value" : sqlalchemy.INT, "id" : sqlalchemy.INT})

## Import Host

In [79]:
host = pd.DataFrame({"host_id" : listings.host_id, "host_since" : listings.host_since, "host_about" : listings.host_about, "host_response_time" : listings.host_response_time, "host_response_rate" : listings.host_response_rate, "host_neighbourhood" : listings.host_neighbourhood, "host_url" : listings.host_url, "host_name" : listings.host_name, "host_thumbnail_url" : listings.host_thumbnail_url, "host_picture_url" : listings.host_picture_url})

In [80]:
host = host.fillna("Unknown")

In [86]:
for i in host.index:
    host.at[i, "host_since"] = datetime.datetime.strptime(str(host.at[i, "host_since"]), "%Y-%m-%d").date()

In [90]:
host = host.drop_duplicates()

In [91]:
host.to_sql("Host", engine, if_exists="append", index=False, dtype={"host_id" : sqlalchemy.CHAR(32), "host_since" : sqlalchemy.DATE, "host_about" : sqlalchemy.VARCHAR(1024), "host_response_time" : sqlalchemy.CHAR(32), "host_response_rate" : sqlalchemy.CHAR(32), "host_neighbourhood" : sqlalchemy.CHAR(32), "host_url" : sqlalchemy.CHAR(32), "host_name" : sqlalchemy.CHAR(32), "host_thumbnail_url" : sqlalchemy.CHAR(32), "host_picture_url" : sqlalchemy.CHAR(32)})

##### We need to add host_neighbourhood, city and country, infer it from the general dataset

In [93]:
listings

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,18666,https://www.airbnb.com/rooms/18666,Flat with Sunny Terrace,"Apartment located near the ""Plaza de las Glori...",Nice apartment situated on the penthouse floor...,"Apartment located near the ""Plaza de las Glori...",Apartment in Barcelona near to the Plaza de la...,NaN,"Good transports connections, 50 m from the met...",Free Wifi - air conditioning. We will provide ...,...,10.0,10.0,2.0,10.0,10.0,8.0,f,flexible,f,f
1,18674,https://www.airbnb.com/rooms/18674,Huge flat for 8 people close to Sagrada Familia,110m2 apartment to rent in Barcelona. Located ...,Apartment with 110 m2 located in the 6th floor...,110m2 apartment to rent in Barcelona. Located ...,Apartment in Barcelona located in the heart of...,NaN,"Good transport connection, 150 m from metro “...",Free Wifi - air conditioning. We will provide ...,...,9.0,10.0,10.0,10.0,9.0,9.0,f,strict_14_with_grace_period,f,f
2,21605,https://www.airbnb.com/rooms/21605,Nice and sunny duble room,"The flat is in Poblenou district, and the room...","L'apartament està al barri de Poblenou, i l'ha...","The flat is in Poblenou district, and the room...",Poblenou as one of the few areas that has grow...,"Tingues en compte que vivim amb gats, assegura...","By metro, tram, bus, taxi, bicycle. See more i...",The kitchen is fully equipped and can use the ...,...,9.0,9.0,10.0,10.0,9.0,9.0,f,moderate,f,f
3,25786,https://www.airbnb.com/rooms/25786,NICE ROOM AVAILABLE IN THE HEART OF GRACIA,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,Room available for rent.- PEDRO PEREZ. Share...,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,Solo decir que a menudo ni salgo del barrio. M...,No dudes en perdir una cita para un masaje rel...,Metro con las principales estaciones. A pie. E...,All access with respect. Kitchen facilities ne...,...,9.0,9.0,10.0,10.0,10.0,9.0,f,strict_14_with_grace_period,t,t
4,31377,https://www.airbnb.com/rooms/31377,"Room for 2, Sagrada Famili",The room in 500 m from Sagrada Familia. 3 bran...,"Great location, right next to Sagrada Familia,...",The room in 500 m from Sagrada Familia. 3 bran...,NaN,NaN,From airport L9 to st. Collblanc. From Collbla...,NaN,...,9.0,10.0,10.0,10.0,9.0,9.0,f,strict_14_with_grace_period,f,f
5,31380,https://www.airbnb.com/rooms/31380,"Room for 2-3. Barcelona, Sagrada Famili","Room for 2-3. Great location, right next to S...","Great location, right next to Sagrada Familia,...","Room for 2-3. Great location, right next to S...",NaN,NaN,From airport L9 to st. Collblanc. From Collbla...,NaN,...,9.0,9.0,9.0,9.0,8.0,9.0,f,strict_14_with_grace_period,f,f
6,31823,https://www.airbnb.com/rooms/31823,"Aribau, Friendly Rentals","This apartment has: 4 single beds, 1 double so...","This apartment has: 4 single beds, 1 double so...","This apartment has: 4 single beds, 1 double so...",EIXAMPLE ESQUERRA (LEFT) This area of the Ei...,NaN,Ideal to discover the city either on foot or b...,Travellers will have access to the entire apar...,...,10.0,9.0,9.0,9.0,10.0,9.0,f,strict_14_with_grace_period,f,f
7,31958,https://www.airbnb.com/rooms/31958,"Gaudi Attic 1, Friendly Rentals","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...",The Gracia neighbourhood is one of the oldest ...,NaN,Ideal to discover the city either on foot or b...,Travellers will have access to the entire apar...,...,9.0,9.0,9.0,9.0,9.0,9.0,f,strict_14_with_grace_period,f,f
8,26033978,https://www.airbnb.com/rooms/26033978,3 bedroom apartment in the center of barcelona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,f,f
9,32471,https://www.airb

## Import Provide TODO